In [ ]:
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [ ]:
df = pd.read_csv('BBGM_League_1_all_seasons_Average_Stats.csv')

In [ ]:
pm,ms = df['+/-'].mean(), df['+/-'].std()

In [ ]:
df.columns

In [ ]:
stats_to_est = ['FT','FTA','FG','FGA','ORB','DRB','AST','TOV','STL','Blk','PF','PTS','3P','3PA','BA']
sT = [_ +'p36' for _ in stats_to_est]

In [ ]:
for s in stats_to_est:
    df[s + 'p36'] = 36*df[s]/np.maximum(df['MP'],1e-6)

In [ ]:
dfs = df[df.MP*df.G > 850]
res = sm.RLM(dfs['Ovr'],sm.add_constant(dfs[stats_to_est])).fit()
res.summary()

In [ ]:
plt.scatter(dfs['Ovr'],res.predict(),s=5,alpha=0.5)

In [ ]:
dfs['sOVR'] = res.predict()
dfs['PS'] = dfs['Ovr']/dfs['sOVR']

In [ ]:
plt.style.use('fivethirtyeight')

In [ ]:
plt.plot(dfs[['Age','PS','Ovr','sOVR']].groupby('Age').mean()['PS'])
plt.xlim(20,44)
plt.ylabel('Ovr ÷ sOVR')
plt.xlabel('Age')

In [ ]:
plt.hist(dfs.Age,13,density='True')
plt.xlim(20,44)
plt.xlabel('Age')
plt.ylabel('Density')

In [ ]:
res.params

In [ ]:
#dft = dfs.groupby(['pid','Age']).mean()
df['sOVR'] = res.predict(sm.add_constant(df[stats_to_est]))
dft = df.groupby(['pid','Age']).mean()

In [ ]:
from collections import defaultdict
p_o = defaultdict(lambda: defaultdict(list))
for row in dft[['Pot','Ovr','sOVR']].itertuples():
    p_o[row[0][0]][row[0][1]].append(row[1:])

In [ ]:
p_y = defaultdict(list)
for p,yd in p_o.items():
    for y,s in yd.items():
        if y+1 in yd and y+2 in yd and y+3 in yd:
            X = list(s[0][:-1])
            Y = np.mean([yd[y+i][-1] for i in range(4)])
            p_y[y].append(X + [Y])

In [ ]:
print('4 year value by age\nage [pot ovr]')
for age in sorted(p_y.keys()):
    Xy = np.array(p_y[age])
    if (Xy[:,0] == Xy[:,1]).all():
        Xy = Xy[:,1:]
    res2 = sm.OLS(Xy[:,-1],Xy[:,:-1]).fit()
    print(age,np.round(res2.params,2))